In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import os
from io import StringIO
from constants import au_postcodes_df
from pydantic import BaseModel, ValidationError
from typing import Optional
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

In [10]:

def preprocess_school_names(name):
    # Implement your preprocessing logic here
    # For example, lowercasing, removing common suffixes/prefixes, etc.
    name.lower()
    name = name.replace('primary', '')
    name = name.replace('secondary', '')
    name = name.replace('school', '')
    name = name.replace('college', '')
    name = name.replace('public', '')
    name = name.replace('private', '')
    return name

# Function to apply fuzzy matching
def get_best_match(row):
    name = str(row['suburb'])
    choices = au_postcodes_df[au_postcodes_df['state'] == row['state']]['suburb']
    best_match = process.extractOne(name, choices)
    if not best_match:
        return None
    return best_match[0]  # Returns the best match name


def crawl_school_from_better_education():

    school_types = ['primary', 'secondary']
    states = list(au_postcodes_df['state'].unique())

    # create an empty DataFrame to store the data
    schools_with_score_df = pd.DataFrame(columns=['school', 'suburb', 'state', 'postcode', 'score', 'school_type'])

    for school_type in school_types:
        for state in states:

            print(f"Start scraping  the webpage: {school_type} {state}")

            url = f'https://bettereducation.com.au/school/{school_type}/{state}/{state}_top_{school_type}_schools.aspx'
            response = requests.get(url)

            if response.status_code == 200:
                # Parse the HTML content
                soup = BeautifulSoup(response.text, 'html.parser')
                table = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_GridView1'})
                table_io = StringIO(str(table))

                # Convert the table to a DataFrame
                df = pd.read_html(table_io)[0]
                
                # Clean the DataFrame
                if state == 'VIC':
                    df = df.rename(columns={'Postcode': 'postcode'})
                    df['suburb'] = df['School'].str.split(',', expand=True).iloc[:, -3:-2]
                if state == 'ACT' or state == 'TAS':
                    df[['suburb', 'state', 'postcode']] = df['School'].str.split(',', expand=True).iloc[:, -3:]
                if state == 'QLD':
                    df['postcode'] = ''
                    df["suburb"] = df['Locality']
                if state == 'SA' or state == 'NT' or state == 'NSW':
                    df[['suburb', 'state', 'postcode']] = df['Locality'].str.split(',', expand=True).iloc[:, 0:3]
                if state == 'WA':
                    if school_type == 'primary':
                        df[['suburb', 'state', 'postcode']] = df['Locality'].str.split(',', expand=True).iloc[:, 0:3]
                    if school_type == 'secondary':
                        df = df.rename(columns={'Postcode': 'postcode'})
                        df['suburb'] = df['School'].str.split(',', expand=True).iloc[:, -3:-2]
                
                #handle special cases
                if school_type == 'primary':
                    if state == 'NSW':
                        df.loc[df['School'] == 'Redlands,Cremorne,NSW,2090', ['suburb', 'state', 'postcode']] = ['Cremorne', 'NSW', '2090']
                    if state == 'NT':
                        df.loc[df['Locality'] == 'Berrimah,NT,828', 'postcode'] = '0828'
                if school_type == 'secondary':
                    if state == 'NSW':
                        df.loc[df['Locality'] == 'Surry Hills NSW 2010', ['suburb', 'state', 'postcode']] = ['Surry Hill', 'NSW', '2010']
                        df.loc[df['Locality'] == 'St Ives,St Ives,NSW,2075', ['suburb', 'state', 'postcode']] = ['St Ives', 'NSW', '2075']
                        df.loc[df['School'] == 'Redlands,Cremorne,NSW,2090', ['suburb', 'state', 'postcode']] = ['Cremorne', 'NSW', '2090']


                df['School'] = df['School'].str.split(',', expand=True).iloc[:, 0]
                df["state"] = state 
                df = df[['School', 'suburb', 'state', 'postcode', 'State Overall Score']]
                df = df.rename(columns={'School': 'school', 'State Overall Score': 'score'})
                df['educationLevel'] = school_type

                # assign the dataframe name df_school_type
                schools_with_score_df = pd.concat([schools_with_score_df, df], ignore_index=True)
                
                print(f"Successfully retrieved the webpage: {school_type} {state}")
            else:
                print(f"Failed to retrieve the webpage: status code {response.status_code}")
            
    return schools_with_score_df


def extract_school_from_suburb_profile():
    directory = os.path.join("D:\\aus_real_estate_data", 'suburb-profile')
    states = os.listdir(directory)
    url = 'http://localhost:8000/schools' # backend url to post data

    schools_df = pd.DataFrame(columns=['school', 'suburb', 'state', 'postcode', 'schoolType', 'educationLevel', 'score'])

    for state_code in states:
        d2 = os.path.join(directory, state_code)
        suburbs = os.listdir(d2)
        
        for suburb in suburbs:
            d3 = os.path.join(d2, suburb)
            json_files = os.listdir(d3)

            # Add missing import for json module

            for json_f in json_files:
                with open(os.path.join(d3, json_f)) as f:
                    suburb_profile = json.load(f)

                try:
                    schools = suburb_profile['props']['pageProps']['details'].get('schoolCatchment', {}).get('schools')
                    # Add a check for the existence of the 'schools' key before accessing it
                    if schools:
                        for school in schools:
                            school_data = SchoolBase(
                                school=school['name'],
                                suburb=" ".join(suburb.split('-')[0:-1]),
                                state=state_code,
                                postcode=str(suburb.split('-')[-1]),
                                schoolType=school['type'],
                                educationLevel=school['educationLevel'],
                                score=None
                            )

                            school_data = school_data.model_dump()
                            if school_data['educationLevel'] == 'combined':
                                school_data['educationLevel'] = 'primary'
                                schools_df = pd.concat([schools_df, pd.DataFrame([school_data])], ignore_index=True)
                                school_data['educationLevel'] = 'secondary'
                                schools_df = pd.concat([schools_df, pd.DataFrame([school_data])], ignore_index=True)
                            else:
                                schools_df = pd.concat([schools_df, pd.DataFrame([school_data])], ignore_index=True)
                except KeyError as e:
                    continue
        
    return schools_df


class SchoolBase(BaseModel):
    school: str
    suburb: str
    state: str
    postcode: str
    schoolType: str
    educationLevel: str
    score: Optional[int] = None


# default header for the request
def header(url):
    headers = {
        'Accept': 'application/json',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6',
        'Cache-Control': 'max-age=0',
        'Referer': url,
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    }
    return headers

In [3]:
schools_df = extract_school_from_suburb_profile()


In [6]:
schools_df.sort_values(by=['postcode'], inplace=True)
schools_df

,school,suburb,state,postcode,schoolType,educationLevel,score
3992,Nemarluk School,ALAWA,NT,0810,Government,primary,None
4079,Nakara Primary School,NAKARA,NT,0810,Government,primary,None
4077,Moil Primary School,MOIL,NT,0810,Government,primary,None
4089,The Essington School,RAPID CREEK,NT,0810,Private,secondary,None
4105,Dripstone Middle School,TIWI,NT,0810,Government,secondary,None
...,...,...,...,...,...,...,...
7539,Strahan Primary School,STRAHAN,TAS,7468,Government,primary,None
7582,Zeehan Primary School,ZEEHAN,TAS,7469,Government,primary,None
7495,St Joseph's Catholic School,ROSEBERY,TAS,7470,Catholic,primary,None
7497,Rosebery District School,ROSEBERY,TAS,7470,Government,secondary,None


In [27]:
schools_with_score_df = crawl_school_from_better_education()

Start scraping  the webpage: primary NT
Successfully retrieved the webpage: primary NT
Start scraping  the webpage: primary NSW
Successfully retrieved the webpage: primary NSW
Start scraping  the webpage: primary ACT
Successfully retrieved the webpage: primary ACT
Start scraping  the webpage: primary VIC
Successfully retrieved the webpage: primary VIC
Start scraping  the webpage: primary QLD
Successfully retrieved the webpage: primary QLD
Start scraping  the webpage: primary SA
Successfully retrieved the webpage: primary SA
Start scraping  the webpage: primary WA
Successfully retrieved the webpage: primary WA
Start scraping  the webpage: primary TAS
Successfully retrieved the webpage: primary TAS
Start scraping  the webpage: secondary NT
Successfully retrieved the webpage: secondary NT
Start scraping  the webpage: secondary NSW
Successfully retrieved the webpage: secondary NSW
Start scraping  the webpage: secondary ACT
Successfully retrieved the webpage: secondary ACT
Start scraping  t

In [13]:
schools_with_score_df

,school,suburb,state,postcode,score,school_type,educationLevel
0,Haileybury Rendall School,Berrimah,NT,0828,100,NaN,primary
1,The Essington School Darwin,Nightcliff,NT,0810,100,NaN,primary
2,Nhulunbuy Christian School,Nhulunbuy,NT,0880,99,NaN,primary
3,Katherine School Of The Air,Katherine,NT,0850,99,NaN,primary
4,Milkwood Steiner School,Berrimah,NT,0828,99,NaN,primary
...,...,...,...,...,...,...,...
2247,Riverside High School,Riverside,TAS,7250,92,NaN,secondary
2248,St Brendan-Shaw College,Devonport,TAS,7310,91,NaN,secondary
2249,Peregrine,Nicholls Rivule,TAS,7112,91,NaN,secondary
2250,Clarence High School,Bellerive,TAS,7018,90,NaN,secondary


In [29]:
QLD_schools_with_score_df = schools_with_score_df[schools_with_score_df['postcode']==""]
QLD_schools_with_score_df['bestmatchsuburb'] = QLD_schools_with_score_df.apply(get_best_match, axis=1)

C:\Users\ericc\AppData\Local\Temp\ipykernel_22152\4174472856.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QLD_schools_with_score_df['bestmatchsuburb'] = QLD_schools_with_score_df.apply(get_best_match, axis=1)


In [30]:
QLD_schools_with_score_df.drop(columns=['postcode', 'suburb'], inplace=True)

C:\Users\ericc\AppData\Local\Temp\ipykernel_22152\4066093252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QLD_schools_with_score_df.drop(columns=['postcode', 'suburb'], inplace=True)


In [32]:
QLD_schools_with_score_df.rename(columns={'bestmatchsuburb': 'suburb'}, inplace=True)

C:\Users\ericc\AppData\Local\Temp\ipykernel_22152\1662873444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QLD_schools_with_score_df.rename(columns={'bestmatchsuburb': 'suburb'}, inplace=True)


In [33]:
QLD_schools_with_score_df

,school,state,score,school_type,educationLevel,suburb
717,Sunnybank Hills State School,QLD,100,NaN,primary,SUNNYBANK HILLS
718,Citipointe Christian College,QLD,100,NaN,primary,CARINDALE
719,Ipswich Grammar School,QLD,100,NaN,primary,IPSWICH
720,St Peters Lutheran College,QLD,100,NaN,primary,INDOOROOPILLY
721,Anglican Church Grammar School,QLD,100,NaN,primary,EAST BRISBANE
...,...,...,...,...,...,...
2070,Iona College,QLD,90,NaN,secondary,WOONDUM
2071,Lutheran Ormeau Rivers District School,QLD,90,NaN,secondary,PIMPAMA
2072,Wynnum State High School,QLD,90,NaN,secondary,MANLY
2073,Ferny Grove State High School,QLD,90,NaN,secondary,FERNY GROVE


In [35]:
au_postcodes_df_copy = au_postcodes_df.copy()
au_postcodes_df_copy.drop_duplicates(subset=['state', 'suburb'], keep="first", inplace=True)

In [36]:
QLD_schools_with_score_df = QLD_schools_with_score_df.merge(au_postcodes_df_copy, on=['suburb', 'state'], suffixes=('_l', '_r'), how='left')
QLD_schools_with_score_df

,school,state,score,school_type,educationLevel,suburb,postcode
0,Sunnybank Hills State School,QLD,100,NaN,primary,SUNNYBANK HILLS,4109
1,Citipointe Christian College,QLD,100,NaN,primary,CARINDALE,4152
2,Ipswich Grammar School,QLD,100,NaN,primary,IPSWICH,4305
3,St Peters Lutheran College,QLD,100,NaN,primary,INDOOROOPILLY,4068
4,Anglican Church Grammar School,QLD,100,NaN,primary,EAST BRISBANE,4169
...,...,...,...,...,...,...,...
514,Iona College,QLD,90,NaN,secondary,WOONDUM,4570
515,Lutheran Ormeau Rivers District School,QLD,90,NaN,secondary,PIMPAMA,4209
516,Wynnum State High School,QLD,90,NaN,secondary,MANLY,4179
517,Ferny Grove State High School,QLD,90,NaN,secondary,FERNY GROVE,4055


In [37]:
schools_with_score_df = schools_with_score_df[['school', 'suburb', 'state', 'postcode', 'score', 'school_type', 'educationLevel']]
QLD_schools_with_score_df = QLD_schools_with_score_df[['school', 'suburb', 'state', 'postcode', 'score', 'school_type', 'educationLevel']]
schools_with_score_df = pd.concat([schools_with_score_df[schools_with_score_df['state']!='QLD'], QLD_schools_with_score_df], ignore_index=True)

In [38]:
schools_with_score_df

,school,suburb,state,postcode,score,school_type,educationLevel
0,Haileybury Rendall School,Berrimah,NT,0828,100,NaN,primary
1,The Essington School Darwin,Nightcliff,NT,0810,100,NaN,primary
2,Nhulunbuy Christian School,Nhulunbuy,NT,0880,99,NaN,primary
3,Katherine School Of The Air,Katherine,NT,0850,99,NaN,primary
4,Milkwood Steiner School,Berrimah,NT,0828,99,NaN,primary
...,...,...,...,...,...,...,...
2247,Iona College,WOONDUM,QLD,4570,90,NaN,secondary
2248,Lutheran Ormeau Rivers District School,PIMPAMA,QLD,4209,90,NaN,secondary
2249,Wynnum State High School,MANLY,QLD,4179,90,NaN,secondary
2250,Ferny Grove State High School,FERNY GROVE,QLD,4055,90,NaN,secondary


In [39]:
schools_with_score_df['school_cleaned'] = schools_with_score_df['school'].apply(preprocess_school_names)
schools_df['school_cleaned'] = schools_df['school'].apply(preprocess_school_names)

In [43]:
schools_with_score_df['BestMatchName'] = schools_with_score_df.apply(get_best_match, axis=1)

In [44]:
schools_with_score_df

,school,suburb,state,postcode,score,school_type,educationLevel,school_cleaned,BestMatchName
0,Haileybury Rendall School,Berrimah,NT,0828,100,NaN,primary,Haileybury Rendall School,BERRIMAH
1,The Essington School Darwin,Nightcliff,NT,0810,100,NaN,primary,The Essington School Darwin,NIGHTCLIFF
2,Nhulunbuy Christian School,Nhulunbuy,NT,0880,99,NaN,primary,Nhulunbuy Christian School,NHULUNBUY
3,Katherine School Of The Air,Katherine,NT,0850,99,NaN,primary,Katherine School Of The Air,KATHERINE
4,Milkwood Steiner School,Berrimah,NT,0828,99,NaN,primary,Milkwood Steiner School,BERRIMAH
...,...,...,...,...,...,...,...,...,...
2247,Iona College,WOONDUM,QLD,4570,90,NaN,secondary,Iona College,WOONDUM
2248,Lutheran Ormeau Rivers District School,PIMPAMA,QLD,4209,90,NaN,secondary,Lutheran Ormeau Rivers District School,PIMPAMA
2249,Wynnum State High School,MANLY,QLD,4179,90,NaN,secondary,Wynnum State High School,MANLY
2250,Ferny Grove State High School,FERNY GROVE,QLD,4055,90,NaN,secondary,Ferny Grove State High School,FERNY GROVE


In [45]:
schools_with_score_df['similarity'] = schools_with_score_df.apply(lambda row: fuzz.ratio(row['school_cleaned'], row['BestMatchName']), axis=1)
schools_with_score_df[(schools_with_score_df.duplicated(subset=['BestMatchName', 'postcode', 'educationLevel'], keep=False)) & (schools_with_score_df['state']=="NSW")].sort_values(by=['BestMatchName', 'postcode', 'educationLevel', 'similarity'], ascending=False)

schools_with_score_df = schools_with_score_df.sort_values(by=['BestMatchName', 'postcode', 'educationLevel', 'similarity'], ascending=False)
schools_with_score_df.drop_duplicates(subset=['BestMatchName', 'postcode', 'educationLevel'], keep='first', inplace=True)
schools_with_score_df = schools_with_score_df[schools_with_score_df['BestMatchName'].notnull()]
schools_with_score_df = schools_with_score_df[schools_with_score_df['similarity'] >= 80]
#concate 3 columns bestmatchname, postcode, educationLevel to create a unique key
schools_with_score_df['key'] = schools_with_score_df['BestMatchName'].astype(str) + schools_with_score_df['postcode'].astype(str) + schools_with_score_df['educationLevel'].astype(str)

schools_df['key'] = schools_df['school_cleaned'].astype(str) + schools_df['postcode'].astype(str) + schools_df['educationLevel'].astype(str)
final = pd.merge(schools_df, schools_with_score_df, how='left', left_on=['key'], right_on=['key'], suffixes=('_l', '_r'))
final[final['school_cleaned_r'].notnull()].drop_duplicates(subset=['school_cleaned_l', 'postcode_l', 'educationLevel_l'], keep='first')

final = final[['school_l', 'suburb_l', 'state_l', 'postcode_l', 'score_r', 'schoolType', 'educationLevel_l']]
final.rename(columns={'school_l': 'school', 'suburb_l': 'suburb', 'state_l': 'state', 'postcode_l': 'postcode', 'score_r': 'score', 'educationLevel_l': 'educationLevel'}, inplace=True)

final.drop_duplicates(subset=['school', 'postcode', 'educationLevel'], keep='first', inplace=True)

In [46]:
final

,school,suburb,state,postcode,score,schoolType,educationLevel
0,Nemarluk School,ALAWA,NT,0810,NaN,Government,primary
1,Nakara Primary School,NAKARA,NT,0810,NaN,Government,primary
2,Moil Primary School,MOIL,NT,0810,NaN,Government,primary
3,The Essington School,RAPID CREEK,NT,0810,NaN,Private,secondary
4,Dripstone Middle School,TIWI,NT,0810,NaN,Government,secondary
...,...,...,...,...,...,...,...
12161,Strahan Primary School,STRAHAN,TAS,7468,NaN,Government,primary
12162,Zeehan Primary School,ZEEHAN,TAS,7469,NaN,Government,primary
12163,St Joseph's Catholic School,ROSEBERY,TAS,7470,NaN,Catholic,primary
12164,Rosebery District School,ROSEBERY,TAS,7470,NaN,Government,secondary
